In [46]:
import csv
from os.path import join
from numpy import array, random, linalg, cross, multiply, where, column_stack, ones_like,zeros_like, sqrt, concatenate,mean
from sklearn.cluster import DBSCAN
import pyransac3d

In [47]:
def points_reader(file_path: str):
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for x,y,z in reader:
            yield float(x),float(y),float(z)

In [48]:
working_dir = R'D:\Programy\Documents\II_EiT\POI'

vertical_file_path = join(working_dir, 'vertical.xyz')
horizontal_file_path = join(working_dir, 'horizontal.xyz')
cylinder_file_path = join(working_dir, 'cylinder.xyz')
    
vertical_points = array([x for x in points_reader(vertical_file_path)])
horizontal_points = array([x for x in points_reader(horizontal_file_path)])
cylinder_points = array([x for x in points_reader(cylinder_file_path)])
all_clouds = concatenate([vertical_points, horizontal_points, cylinder_points])

In [49]:
dbscan = DBSCAN(eps=40, min_samples=800).fit(all_clouds).labels_

points0 =[]
points1=[]
points2=[]
for i in range(len(dbscan)):
    if dbscan[i] == 0:
        points0.append(all_clouds[i])
    elif dbscan[i] == 1:
        points1.append(all_clouds[i])
    elif dbscan[i] == 2:
        points2.append(all_clouds[i])
points0 = array(points0)
points1 = array(points1)
points2 = array(points2)

In [50]:
def distance_point_plane(abcd,xyz):

    numerator = abs(abcd[0] * xyz[0] + abcd[1] * xyz[1] + abcd[2] * xyz[2] + abcd[3])
    denominator = sqrt(abcd[0]**2+abcd[1]**2+abcd[2]**2)
    return numerator / denominator


plane = pyransac3d.Plane()

In [51]:
(normal_vector, inliers) = plane.fit(points0, thresh=0.01, maxIteration=1000)  # Poprawione przypisanie
A,B,C,D = normal_vector
# Wektor normalny płaszczyzny
normal_vector0 = array([A, B, C])
normal_vector0 /= linalg.norm(normal_vector)  # Normalizacja


distance0=[]
for i in range(len(points0)):
    distance0.append(distance_point_plane([A,B,C,D], points0[i]))
distance0 = mean(distance0)



In [52]:
(normal_vector, inliers) = plane.fit(points1, thresh=0.01, maxIteration=1000)  # Poprawione przypisanie
A,B,C,D = normal_vector
# Wektor normalny płaszczyzny
normal_vector1 = array([A, B, C])
normal_vector1 /= linalg.norm(normal_vector)  # Normalizacja


distance1=[]
for i in range(len(points1)):
    distance1.append(distance_point_plane([A,B,C,D], points1[i]))
distance1 = mean(distance1)

In [53]:
(normal_vector, inliers) = plane.fit(points2, thresh=0.01, maxIteration=1000)  # Poprawione przypisanie
A,B,C,D = normal_vector
# Wektor normalny płaszczyzny
normal_vector2 = array([A, B, C])
normal_vector2 /= linalg.norm(normal_vector)  # Normalizacja


distance2=[]
for i in range(len(points2)):
    distance2.append(distance_point_plane([A,B,C,D], points2[i]))
distance2 = mean(distance2)

In [56]:
print("Wektor normalny punktow 0:", normal_vector0)
print("Wektor normalny punktow 1:", normal_vector1)
print("Wektor normalny punktow 2:", normal_vector2)

if distance0>distance1 and distance0>distance2:
    print("Chmura 0 nie jest płaszczyzną")
    if normal_vector1[2]>normal_vector1[0] and normal_vector1[2]>normal_vector1[1]:
        print("Płaszczyzna 1 jest pozioma")
    else:
        print("Płaszczyzna 1 jest pionowa")
    if normal_vector2[2]>normal_vector2[0] and normal_vector2[2]>normal_vector2[1]:
        print("Płaszczyzna 2 jest pozioma")
    else:
        print("Płaszczyzna 2 jest pionowa")
if distance1>distance0 and distance1>distance2:
    print("Chmura 1 nie jest płaszczyzną")
    if normal_vector0[2]>normal_vector0[0] and normal_vector0[2]>normal_vector0[1]:
        print("Płaszczyzna 0 jest pozioma")
    else:
        print("Płaszczyzna 0 jest pionowa")
    if normal_vector2[2]>normal_vector2[0] and normal_vector2[2]>normal_vector2[1]:
        print("Płaszczyzna 2 jest pozioma")
    else:
        print("Płaszczyzna 2 jest pionowa")
if distance2>distance1 and distance2>distance0:
    print("Chmura 2 nie jest płaszczyzną")
    if normal_vector0[2]>normal_vector0[0] and normal_vector0[2]>normal_vector0[1]:
        print("Płaszczyzna 0 jest pozioma")
    else:
        print("Płaszczyzna 0 jest pionowa")
    if normal_vector1[2]>normal_vector1[0] and normal_vector1[2]>normal_vector1[1]:
        print("Płaszczyzna 1 jest pozioma")
    else:
        print("Płaszczyzna 1 jest pionowa")

Wektor normalny punktow 0: [-1.32551160e-05 -9.93714923e-02  2.50118742e-05]
Wektor normalny punktow 1: [-4.40184272e-04 -5.34364052e-06 -9.99519021e-01]
Wektor normalny punktow 2: [-2.87120342e-01 -3.21688114e-02  1.15286669e-04]
Chmura 2 nie jest płaszczyzną
Płaszczyzna 0 jest pozioma
Płaszczyzna 1 jest pionowa
